# Figure 6

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import numpy as np
import matplotlib.pylab as plt
import delfi.distribution as dd
import time
from copy import deepcopy
import sys
sys.path.append("../../../setup")
sys.path.append("../../../simulator")
sys.path.append("../../../inference")
sys.path.append("../../../utils")
sys.path.append("../model/")
sys.path.append('../../')

import netio
import viz
from compare_samples import get_dist, evaluate_summ_stats
from find_pyloric import merge_samples, params_are_bounded
import dill as pickle
import matplotlib as mpl
from find_pyloric import check_ss_to_obs
import helpers

from common import col, svg, samples_nd, get_labels_8pt
from conditional_density import eval_conditional_density, conditional_correlation, conditional_mutual_information, buildCondCovMatrix, partialCorrelation
import matplotlib.colors as mcolors
import energy as ue
import torch

ModuleNotFoundError: No module named 'viz'